In [2]:
!pip install faker &> /dev/null

In [3]:
import pandas as pd
import numpy as np
from faker import Faker
from faker.providers import *
from itertools import product
import random
import json, math
import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn3_circles, venn2
fake = Faker()  # initialise faker module

In [4]:
def create_data_faker(num=1):
    output = [{
        "Negotiating Users": fake.simple_profile()['username']
    }
        for x in range(num)
    ]
    return output

def create_data_faker_2_users(num=1):
    output = [{
        "User1": fake.simple_profile()['username'],
        "User2": fake.simple_profile()['username']
    }
        for x in range(num)
    ]
    return output

def calculate_pr_sl(beta, sensi, alpha, lambd=0.5):
    PR = 0
    for (i, j) in zip(beta, sensi):
        PR = PR+(i*j) * alpha  # calculating privacy risk
    print("PR:", PR)

    SL = 0
    for(i, j) in zip(beta, sensi):
        SL = SL+((1-i)*(1-j) * alpha)  # calculating sharing loss
    print("SL:", SL)
    sum = 0
    for i in sensi:
        sum = sum+i
    # lambd = 0.209725  # try different values for lambda to see a difference in results
    lambd = 5  # 0.5 gives an equal weightage for privacy risk and sharing loss
    # print(lambd*PR) # print values to identify whether imp of privacy risk is higher
    # print((1-lambd)*SL) # or sharing loss

    ##############################################################
    # modify conflict resolution to accomodate soi, reputation, il
    ##############################################################

    if(lambd*PR > ((10-lambd)*SL)):  # conflict resolution
        return "DENY"
    else:
        return "PERMIT"

def return_pr_sl(beta, sensi, alpha, lambd=0.5):
    PR = 0
    for (i, j) in zip(beta, sensi):
        PR = PR+(i*j) * alpha  # calculating privacy risk
    print("PR:", PR)

    SL = 0
    for(i, j) in zip(beta, sensi):
        SL = SL+((1-i)*(1-j) * alpha)  # calculating sharing loss
    print("SL:", SL)
    sum = 0
    for i in sensi:
        sum = sum+i
    # lambd = 0.209725  # try different values for lambda to see a difference in results
    lambd = 0.5  # 0.5 gives an equal weightage for privacy risk and sharing loss
    # print(lambd*PR) # print values to identify whether imp of privacy risk is higher
    # print((1-lambd)*SL) # or sharing loss
    
    return PR, SL

In [5]:
uname = []
uname1 = []
uname2 = []
sentiment_val = []
bot_not = []
reputation = []
identity_leakage = []
final = []
num = 50
for x in range(num):
  uname1.append(fake.simple_profile()['username'])

print(uname1[1])

for x in range(num):
  uname = str(uname1[x])
  for y in range(num):
    uname2.append(fake.simple_profile()['username'])
    sentiment_val.append(fake.pydecimal(left_digits=1, right_digits=3, positive=False, min_value=0, max_value=1))
    bot_not.append(fake.boolean(random.randint(0, math.ceil(num/80))))
    reputation.append(fake.pydecimal(left_digits=1, right_digits=3, positive=False, min_value=0, max_value=1))
    identity_leakage.append(fake.pydecimal(left_digits=1, right_digits=3, positive=False, min_value=0, max_value=1))
    final.append(uname)
  
zipped = list(zip(final, uname2, sentiment_val, bot_not, reputation, identity_leakage))
df_supp = pd.DataFrame(zipped, columns = ['Username1', 'Username2', 'Nature of Interaction', 'Bot or Not', 'Reputation', 'Identity Leakage'])
df_supp

# zipped = list(zip(sentiment_val, bot_not, reputation, identity_leakage))
# df_supp = pd.DataFrame(zipped, columns = ['Nature of Interaction', 'Bot or Not', 'Reputation', 'Identity Leakage'])
# df_uname = pd.DataFrame(create_data_faker(num))
# prod = product(df_uname.values, df_supp.values)
# df_prod = pd.DataFrame(list(prod))

# df_prod.head(100)

davidwilliams


,Username1,Username2,Nature of Interaction,Bot or Not,Reputation,Identity Leakage
0,michael06,vaughanstephanie,0.547,False,0.442,1
1,michael06,cfowler,1,False,1,0.416
2,michael06,sparker,0.228,False,0.790,0.309
3,michael06,brandon01,1,False,0.769,1
4,michael06,murphyyolanda,1,False,0.973,0.858
...,...,...,...,...,...,...
2495,kevinsimpson,gabrielle24,0.743,False,0.827,0.504
2496,kevinsimpson,awilliams,1,False,1,1
2497,kevinsimpson,richard43,0.495,False,1,0.857
2498,kevinsimpson,xbrown,0.718,False,0.354,0.422


In [6]:
uname = []
uname1 = []
uname2 = []
final = []
sentiment_val = []
identity_leakage = []
type_interaction = []
num = 50
for x in range(num):
  uname1.append(fake.simple_profile()['username'])

for x in range(num):
  uname = str(uname1[x])
  for y in range(num):
    uname2.append(fake.simple_profile()['username'])
    sentiment_val.append(fake.pydecimal(left_digits=1, right_digits=3, positive=False, min_value=0, max_value=1))
    type_interaction.append(random.randint(0, 3))
    identity_leakage.append(fake.pydecimal(left_digits=1, right_digits=3, positive=False, min_value=0, max_value=1))
    final.append(uname)

zipped = list(zip(final, uname2, sentiment_val, type_interaction, identity_leakage))
df_soi1 = pd.DataFrame(zipped, columns = ['Username1', 'Username2', 'Nature of Interaction', 'Type of Interaction', 'Identity Leakage'])
print(df_soi1.head(100))

# df_supp['Nature of Interaction'] = df_supp['Nature of Interaction'].astype('int')
# df_supp['Identity Leakage'] = df_supp['Identity Leakage'].astype('int')
# df_soi_calc = pd.DataFrame(create_data_faker_2_users(num))
# prod = product(df_soi_calc.values, df_supp.values)
# df_soi = pd.DataFrame(list(prod))
#df_soi.info()
#df_soi1 = pd.DataFrame(df_soi[1].to_list(), columns=['n', 't', 'il'])

soi1 = []
#likes between 2 users
# for i in range(0,2499,50):
#   likes = 0
#   comments = 0 
#   shares = 0
#   posts = 0
#   tot_strength = 0
#   j1 = []
#   for j in range(i,i+50):
#     if df_soi1['n'][j] == 0:
#       likes = likes + df_soi1['t'][j] * 1
#     elif df_soi1['n'][j] == 1:
#       comments = comments + df_soi1['t'][j] * 3
#     elif df_soi1['n'][j] == 2:
#       shares = shares + df_soi1['t'][j] * 5
#     elif df_soi1['n'][j] == 3:
#       posts = posts + df_soi1['t'][j] * 10 
#     # j1.append(j)
#     tot_strength = likes + comments + shares + posts
#     # print("Total strength for i = ", i, " and j = ", j, " is = ", tot_strength)
#   soi1.append(tot_strength)
#   # print(j1)   
# print(soi1)
# print(len(soi1))
# df_soi1.head(100)

          Username1           Username2 Nature of Interaction  \
0   angeladickerson            rjohnson                     1   
1   angeladickerson        aprilsanders                 0.064   
2   angeladickerson          anaramirez                     1   
3   angeladickerson           kiddkatie                     1   
4   angeladickerson          joshuarice                 0.788   
..              ...                 ...                   ...   
95    smithmichelle  elizabethjefferson                     1   
96    smithmichelle           zachary01                 0.080   
97    smithmichelle              lcantu                 0.844   
98    smithmichelle           pgriffith                     1   
99    smithmichelle              gina27                 0.824   

    Type of Interaction Identity Leakage  
0                     1                1  
1                     1                1  
2                     2            0.662  
3                     2                1  
4   

In [17]:
from google.colab import files
uploaded = files.upload()


Saving dataset_5.csv to dataset_5 (1).csv


In [18]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['dataset_5.csv']))
print(df)


             PERSON_ID             NAME        HELLO    DURATION  \
0    PERSON_0000000000  Matthew Chapman  hello world   14.507465   
1    PERSON_0000000001  Jennifer Martin  hello world   94.979314   
2    PERSON_0000000002   Nicholas Ochoa  hello world   85.295803   
3    PERSON_0000000003     Bryce Finley  hello world   43.478226   
4    PERSON_0000000004      Justin Barr  hello world   62.491785   
..                 ...              ...          ...         ...   
235  PERSON_0000000000  Matthew Chapman  hello world  145.859719   
236  PERSON_0000000001  Jennifer Martin  hello world  263.859718   
237  PERSON_0000000002   Nicholas Ochoa  hello world   59.257127   
238  PERSON_0000000003     Bryce Finley  hello world  233.014165   
239  PERSON_0000000004      Justin Barr  hello world   45.374861   

                INTERACTION     COUNTERPART_ID COUNTER_PART_NAME  \
0    INTERACTION_0000000000  PERSON_0000000003      Bryce Finley   
1    INTERACTION_0000000001  PERSON_0000000000 

In [19]:
df['sentiment'] = df['P(POSITIVE)']-df['P(NEGATIVE)']
#normalising sentiment values b/w [0,1]
df['normalized_sentiment'] = (df['sentiment'] - df['sentiment'].min())/(df['sentiment'].max() - df['sentiment'].min())
wi = []
#multiplying it by the weights for different interactions
for index, row in df.iterrows():
    if row['INTERACTION'] == 'INTERACTION_0000000001':
      val = row['normalized_sentiment'] * 3
    elif row['INTERACTION'] == 'INTERACTION_0000000002':
      val = row['normalized_sentiment'] * 5
    elif row['INTERACTION'] == 'INTERACTION_0000000003':
      val = row['normalized_sentiment'] * 10
    else:
      val = row['normalized_sentiment']
    wi.append(val)
#adding column after multiplying by the weights   
df['weighted_interaction'] = wi
df.head()
persons = ['PERSON_0000000000','PERSON_0000000001', 'PERSON_0000000002','PERSON_0000000003','PERSON_0000000004']
#calculating soi for different interaction pairs, for eg, ab all of a's interactions with b 
soi= {}
for index, row in df.iterrows():
  if row['PERSON_ID'] in persons and row['PERSON_ID'] != row['COUNTERPART_ID']:
    person, counterpart = row['PERSON_ID'], row['COUNTERPART_ID']
    ind = str(persons.index(person)) + str(persons.index(counterpart)) 
    if soi.get(ind) != None:
      soi[ind] += row['weighted_interaction']
    else:
      soi[ind] =0
print(soi)




{'03': 14.507537688442213, '10': 30.79899497487437, '23': 72.31658291457286, '34': 53.41206030150754, '02': 26.39195979899497, '43': 36.231155778894475, '21': 41.94974874371859, '01': 12.874371859296483, '14': 8.216080402010052, '13': 24.035175879396984, '04': 8.693467336683417}


In [22]:
def normalize(soi, target=1.0):
   raw = sum(soi.values())
   factor = target/raw
   return {key:value*factor for key,value in soi.items()}

normalized_soi = normalize(soi)
print(normalized_soi)
#Matthew is 2
#Jennifer is 1
#Nicholas is 3 

# soi for Matthew and Jennifer is 01+10
soi_jm = (normalized_soi['21'] + 0)/2 #'12' is 0
soi_nm = (normalized_soi['23'] + 0)/2 #'32' is 0
print(soi_jm, soi_nm)
soi_threshold = (soi_jm + soi_nm)/2

if soi_jm > soi_threshold:
  lambd = lambd + (soi_jm - soi_threshold)
else:
  lambd = lambd - (soi_jm - soi_threshold)
def return_soi(soi_vals):
  return normalized_soi['21'], normalized_soi['23']

return_soi(normalized_soi)
return_pr_sl([0.66, 0.66], [0.5,0.33], 1, lambd=0.6 )
calculate_pr_sl([0.66, 0.66], [0.5,0.33], 1, lambd=0.6)

{'03': 0.04403868448349503, '10': 0.09349258649093903, '23': 0.21952224052718283, '34': 0.16213618890719383, '02': 0.08011471108670448, '43': 0.10998230520471047, '21': 0.12734150954908777, '01': 0.039081090975654396, '14': 0.024940508877905915, '13': 0.0729605223015437, '04': 0.0263896515955824}
0.06628683873329672 0.08011471108670448
PR: 0.5478000000000001
SL: 0.39779999999999993
PR: 0.5478000000000001
SL: 0.39779999999999993


'DENY'